## imports

In [1]:
import wmfdata as wmf
import pandas as pd
import numpy as np

In [2]:
import json
import re
import warnings

In [3]:
pd.options.display.max_columns = None
bold = '\033[1m'
end = '\033[0m'

## spark_session

In [4]:
spark_session = wmf.spark.get_active_session()

if type(spark_session) != type(None):
    spark_session.stop()
else:
    print('no active session')

no active session


In [4]:
spark_session = wmf.spark.create_custom_session(
    master="yarn",
    app_name='bot-vandal-reverts',
    spark_config={
        "spark.driver.memory": "4g",
        "spark.dynamicAllocation.maxExecutors": 64,
        "spark.executor.memory": "20g",
        "spark.executor.cores": 4,
        "spark.sql.shuffle.partitions": 256,
        "spark.driver.maxResultSize": "2g"
        
    }
)

SPARK_HOME: /usr/lib/spark3
Using Hadoop client lib jars at 3.2.0, provided by Spark.
PYSPARK_PYTHON=/opt/conda-analytics/bin/python3


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/21 08:22:38 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/07/21 08:22:38 WARN Utils: Service 'sparkDriver' could not bind on port 12000. Attempting port 12001.
23/07/21 08:22:38 WARN Utils: Service 'sparkDriver' could not bind on port 12001. Attempting port 12002.
23/07/21 08:22:38 WARN Utils: Service 'sparkDriver' could not bind on port 12002. Attempting port 12003.
23/07/21 08:22:38 WARN Utils: Service 'sparkDriver' could not bind on port 12003. Attempting port 12004.
23/07/21 08:22:38 WARN Utils: Service 'sparkDriver' could not bind on port 12004. Attempting port 12005.
23/07/21 08:22:38 WARN Utils: Service 'sparkDriver' could not bind on port 12005. Attempting port 12006.
23/07/21 08:22:38 WARN Utils: Service 'spar

In [5]:
# ig_warn()
# spark_session = wmf.spark.create_session(type='yarn-large')
spark_session

In [6]:
spark_session.sparkContext.setLogLevel("ERROR")

## data

In [7]:
bots = {
    'enwiki': 'ClueBot NG',
    'eswiki': 'SeroBOT',
    'frwiki': 'Salebot',
    'ptwiki': 'Salebot',
    'fawiki': 'Dexbot',
    'bgwiki': 'PSS 9',
    'simplewiki': 'ChenzwBot',
    'ruwiki': 'Рейму Хакурей',
    'rowiki': 'PatrocleBot'
}

In [26]:
%%time

query = """
WITH
    base AS (
        SELECT
            wiki_db,
            revision_id,
            event_timestamp,
            page_namespace,
            revision_first_identity_reverting_revision_id,
            revision_seconds_to_identity_revert
        FROM
            wmf.mediawiki_history
        WHERE
            snapshot = '2023-06' AND
            wiki_db IN {DBS} AND
            event_entity = 'revision' AND
            event_type = 'create' AND
            revision_is_identity_reverted),
            
    bot_reverts AS (
        SELECT
            *
            
        FROM
            wmf.mediawiki_history mwh
            JOIN base ON mwh.revision_id = base.revision_first_identity_reverting_revision_id
        WHERE
            snapshot = '2023-06' AND
            event_user_text IN {BOTS} AND
            ((YEAR(base.event_timestamp) = 2022 AND MONTH(base.event_timestamp) > 6) OR
            (YEAR(base.event_timestamp) = 2023 AND MONTH(base.event_timestamp) <= 6))
        )
        
SELECT *
FROM bot_reverts
LIMIT 20
"""

result = wmf.spark.run(query.format(DBS=wmf.utils.sql_tuple(bots.keys()),
                                   BOTS=wmf.utils.sql_tuple(set(bots.values()))))

23/07/21 10:23:53 ERROR YarnScheduler: Lost executor 200 on an-worker1100.eqiad.wmnet: Container killed by YARN for exceeding physical memory limits. 17.7 GB of 17.6 GB physical memory used. Consider boosting spark.executor.memoryOverhead.
23/07/21 10:24:26 ERROR YarnScheduler: Lost executor 168 on an-worker1109.eqiad.wmnet: Container killed by YARN for exceeding physical memory limits. 18.0 GB of 17.6 GB physical memory used. Consider boosting spark.executor.memoryOverhead.
23/07/21 10:24:51 ERROR YarnScheduler: Lost executor 180 on an-worker1109.eqiad.wmnet: Container killed by YARN for exceeding physical memory limits. 17.7 GB of 17.6 GB physical memory used. Consider boosting spark.executor.memoryOverhead.
23/07/21 10:24:53 ERROR YarnScheduler: Lost executor 216 on an-worker1097.eqiad.wmnet: Container killed by YARN for exceeding physical memory limits. 18.2 GB of 17.6 GB physical memory used. Consider boosting spark.executor.memoryOverhead.
23/07/21 10:24:59 ERROR YarnScheduler: L

In [ ]:
result.head(5)

In [ ]:

result.shape